In [ ]:
#Import libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os

In [ ]:
#Mount google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Load dataset

In [ ]:
img_height, img_width = 224, 224
batch_size = 32

train_dir = "/content/drive/MyDrive/Tumour-20250719T154752Z-1-001/Tumour/train"
val_dir = "/content/drive/MyDrive/Tumour-20250719T154752Z-1-001/Tumour/valid"

train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    zoom_range=0.2,
    shear_range=0.2,
    rotation_range=10
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

val_data = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

In [ ]:
#Build custom CNN model

In [ ]:
# Define input shape
input_shape = (img_height, img_width, 3)

# Define number of tumor categories
num_classes = train_data.num_classes

# Build the model
cnn_model = Sequential()

# Block 1
cnn_model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape))
cnn_model.add(BatchNormalization())
cnn_model.add(MaxPooling2D(pool_size=(2, 2)))

# Block 2
cnn_model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
cnn_model.add(BatchNormalization())
cnn_model.add(MaxPooling2D(pool_size=(2, 2)))
cnn_model.add(Dropout(0.3))

# Block 3
cnn_model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
cnn_model.add(BatchNormalization())
cnn_model.add(MaxPooling2D(pool_size=(2, 2)))
cnn_model.add(Dropout(0.4))

# Flatten and Fully Connected Layers
cnn_model.add(Flatten())
cnn_model.add(Dense(128, activation='relu'))
cnn_model.add(BatchNormalization())
cnn_model.add(Dropout(0.5))

# Output layer
cnn_model.add(Dense(num_classes, activation='softmax'))

cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

cnn_model.summary()

In [ ]:
#Train custom CNN

In [ ]:
# Define Callbacks
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

checkpoint = ModelCheckpoint(
    filepath='best_custom_cnn_model.h5',
    save_best_only=True,
    monitor='val_accuracy',
    verbose=1
)

# Train the model
history_cnn = cnn_model.fit(
    train_data,
    epochs=5,
    validation_data=val_data,
    callbacks=[early_stop, checkpoint]
)

In [ ]:
#Transfer Learning with ResNet50

In [ ]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
base_model.trainable = False  # Freeze base layers

model_resnet = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(train_data.num_classes, activation='softmax')
])

model_resnet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_resnet.summary()

history_resnet = model_resnet.fit(
    train_data,
    validation_data=val_data,
    epochs=5,
    callbacks=[early_stop, ModelCheckpoint("best_resnet_model.h5", save_best_only=True)]
)

In [ ]:
#Model Evaluation

In [ ]:
# Get true labels and predicted labels
val_data.reset()
y_true = val_data.classes
y_pred_probs = model_resnet.predict(val_data)
y_pred = np.argmax(y_pred_probs, axis=1)

# Get class labels
class_labels = list(val_data.class_indices.keys())

report = classification_report(y_true, y_pred, target_names=class_labels)
print("Classification Report:\n", report)

cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()

In [ ]:
def plot_history(history, title='Model'):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs_range = range(1, len(acc) + 1) # Corrected to get the number of epochs from history

    plt.figure(figsize=(8, 4))

    # Accuracy plot
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Train Accuracy', color='green')
    plt.plot(epochs_range, val_acc, label='Val Accuracy', color='orange')
    plt.title(f'{title} - Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    # Loss plot
    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Train Loss', color='blue')
    plt.plot(epochs_range, val_loss, label='Val Loss', color='red')
    plt.title(f'{title} - Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.tight_layout()
    plt.show()


plot_history(history_resnet, title='ResNet50')
plot_history(history_cnn, title='Custom CNN')

In [ ]:
#Model Comparison

In [ ]:
val_loss, val_acc = model_resnet.evaluate(val_data)
print(f"ResNet50 Validation Accuracy: {val_acc*100:.2f}%")

val_loss_cnn, val_acc_cnn = cnn_model.evaluate(val_data)
print(f"Custom CNN Validation Accuracy: {val_acc_cnn*100:.2f}%")

In [ ]:
# Custom CNN model is the best model

In [ ]:
import os
save_path = '/content/drive/MyDrive/brain_tumor_models'
os.makedirs(save_path, exist_ok=True)

In [ ]:
import shutil
shutil.move('best_custom_cnn_model.h5', f'{save_path}/best_custom_cnn_model.h5')

In [ ]:
# STREAMLIT APP CODE

In [ ]:
!pip install streamlit streamlit_option_menu

In [ ]:
%%writefile Tumor.py
import streamlit as st
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

# Load the trained model
model = load_model(r"/content/drive/MyDrive/brain_tumor_models/best_custom_cnn_model.h5")

# Define class names
class_names = ['glioma', 'meningioma', 'no_tumor', 'pituitary']

st.title("🧠 Brain Tumor MRI Classification")
st.write("📤 Upload an MRI image to predict the tumor type.")

# Upload image
uploaded_file = st.file_uploader("Choose an MRI image...", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:
    st.image(uploaded_file, caption='Uploaded MRI Image', use_container_width=True)

    # Preprocess the image
    img = load_img(uploaded_file, target_size=(224, 224))
    img_array = img_to_array(img)
    img_array = img_array / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Make prediction
    prediction = model.predict(img_array)
    class_index = np.argmax(prediction)
    class_name = class_names[class_index]
    confidence = np.max(prediction)

    st.success(f"🧾 **Prediction**: {class_name} ({confidence * 100:.2f}%)")

In [ ]:
!wget -q -O - ipv4.icanhazip.com

In [22]:
! streamlit run Tumor.py & npx localtunnel --port 8501

  Stopping...
^C
